# テーマ：Fashion-MNISTデータセットを畳み込みニューラルネットワーク（CNN）で学習する
## 制約
* 全体の実行時間は60分以内

## 今回参考にしたもの
* [Get started with TensorFlow 2.0 for experts](https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/quickstart/advanced.ipynb)

In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf

#print(tfds.list_builders())
dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']
#print(info)

In [0]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

batch_size = 32

dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

In [0]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, BatchNormalization, Dropout, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras import Model

class CNNModel(Model):
    def __init__(self):
        super(CNNModel, self).__init__()
        drop_rate = 0.5
        
        self._layers = ([
            Conv2D(32, 3), # 28, 28, 1 -> 26, 26, 32
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(64, 3), # 26, 26, 32 -> 24, 24, 64
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 24, 24, 64 -> 12, 12, 64
            Conv2D(128, 3), # 12, 12, 64 -> 10, 10, 128
            BatchNormalization(),
            Activation(tf.nn.relu),
            Conv2D(256, 3), # 10, 10, 128 -> 8, 8, 256
            BatchNormalization(),
            Activation(tf.nn.relu),
            MaxPool2D(), # 8, 8, 256 -> 4, 4, 256
            Flatten(), # 4, 4, 256 -> 4096
            Dense(256), # 4096 -> 256
            BatchNormalization(),
            Dropout(drop_rate),
            Activation(tf.nn.relu),
            Dense(10, activation = 'softmax') # 256 -> 10                        
        ])                
        
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x
       
    
model = CNNModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [0]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
        
@tf.function
def test_step(image, label):
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)

In [24]:
import time

num_epoch = 10
start_time = time.time()

for epoch in range(num_epoch):
    for image, label in dataset_train:
        train_step(image, label)
        
    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}, spent_time: {} min'
    spent_time = time.time() - start_time
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100, spent_time / 60))

Epoch 1, Loss: 0.42033544182777405, Accuracy: 84.49333190917969, Test Loss: 0.2868334949016571, Test Accuracy: 89.27999877929688, spent_time: 1.1366963028907775 min
Epoch 2, Loss: 0.34110528230667114, Accuracy: 87.41333770751953, Test Loss: 0.27765366435050964, Test Accuracy: 89.69000244140625, spent_time: 2.2080264488855996 min
Epoch 3, Loss: 0.2990953326225281, Accuracy: 88.945556640625, Test Loss: 0.2650168836116791, Test Accuracy: 90.2066650390625, spent_time: 3.274205768108368 min
Epoch 4, Loss: 0.26979804039001465, Accuracy: 90.0370864868164, Test Loss: 0.25918814539909363, Test Accuracy: 90.54750061035156, spent_time: 4.3485806584358215 min
Epoch 5, Loss: 0.24623523652553558, Accuracy: 90.892333984375, Test Loss: 0.2535325586795807, Test Accuracy: 90.86400604248047, spent_time: 5.421491026878357 min
Epoch 6, Loss: 0.22641880810260773, Accuracy: 91.6177749633789, Test Loss: 0.2553942799568176, Test Accuracy: 90.9566650390625, spent_time: 6.492582261562347 min
Epoch 7, Loss: 0.209